Authors: A.Radhakrishnan, GFDL
Ack: Anderson Banihirwe for intake-esm updates, GFDL colleagues for data

In [1]:
from netCDF4 import Dataset
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import xarray as xr
import intake,yaml
import intake_esm
import numpy as np
#anonymous True access to test data s3://gfdl-esgf/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Amon/tas/gr1/v20190726
##importing packages needed

In [4]:
col_url = "esgf-world.json" 

In [5]:
col = intake.open_esm_datastore(col_url)

In [6]:
col.df

,project,institute,model,experiment_id,frequency,modeling_realm,mip_table,ensemble_member,grid_label,variable,temporal subset,version,path
0,CMIP6,AS-RCEC,TaiESM1,histSST-piNTCF,NaN,NaN,AERmon,r1i1p1f1,gn,ps,185001-201412,v20200318,s3://esgf-world/CMIP6/AerChemMIP/AS-RCEC/TaiES...
1,CMIP6,AS-RCEC,TaiESM1,histSST-piNTCF,NaN,NaN,CFmon,r1i1p1f1,gn,ta,185001-201412,v20200318,s3://esgf-world/CMIP6/AerChemMIP/AS-RCEC/TaiES...
2,CMIP6,AS-RCEC,TaiESM1,histSST-piNTCF,NaN,NaN,Lmon,r1i1p1f1,gn,rh,185002-201412,v20200318,s3://esgf-world/CMIP6/AerChemMIP/AS-RCEC/TaiES...
3,CMIP6,AS-RCEC,TaiESM1,histSST,NaN,NaN,AERmon,r1i1p1f1,gn,ps,185001-201412,v20200310,s3://esgf-world/CMIP6/AerChemMIP/AS-RCEC/TaiES...
4,CMIP6,AS-RCEC,TaiESM1,histSST,NaN,NaN,CFmon,r1i1p1f1,gn,ta,185001-201412,v20200316,s3://esgf-world/CMIP6/AerChemMIP/AS-RCEC/TaiES...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
301623,CMIP6,THU,CIESM,ssp245,mon,ocean,Omon,r1i1p1f1,gn,thetao,206501-210012,v20200220,s3://esgf-world/CMIP6/ScenarioMIP/THU/CIESM/ss...
301624,CMIP6,THU,CIESM,ssp585,mon,atmos,Amon,r1i1p1f1,gr,pr,201501-210012,v20200417,s3://esgf-world/CMIP6/ScenarioMIP/THU/CIESM/ss...
301625,CMIP6,THU,CIESM,ssp585,mon,atmos,Amon,r1i1p1f1,gr,ta,201501-210012,v20200417,s3://esgf-world/CMIP6/ScenarioMIP/THU/CIESM/ss...
301626,CMIP6,THU,CIESM,ssp585,mon,ocean,Omon,r1i1p1f1,gn,thetao,201501-206412,v20200220,s3://esgf-world/CMIP6/ScenarioMIP/THU/CIESM/ss...


In [ ]:
dset_dict = col.to_dataset_dict(cdf_kwargs={'chunks': {'time': 20}}, storage_options={'anon':True})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'project.institute.model.experiment_id.mip_table'


/Users/ar46/opt/anaconda3/lib/python3.7/site-packages/xarray/conventions.py:500: SerializationWarning: variable 'wtem' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  decode_timedelta=decode_timedelta,


In [16]:
#Alternate way to just search for what we want expname_filter = ['historical']
#table_id_filter = 'Amon'
#model_filter = 'GFDL-ESM4'
#variable_id_filter = "tas"
#cat = col.search(experiment=expname_filter, mip_table=table_id_filter,model=model_filter)

In [7]:
hxr_gfdl_esm4= dset_dict["output.NOAA-GFDL.GFDL-ESM4.historical.Amon"]
fxr_gfdl_esm4= dset_dict["output.NOAA-GFDL.GFDL-ESM4.ssp370.Amon"]


#fxr = xr.open_dataset(file_future)
#hxr = xr.open_dataset(file_hist)

NameError: name 'dset_dict' is not defined

! GLOBAL MEAN 
let hgtas = tas[x=@ave,y=@ave,d=1,l=529:780@ave]


In [7]:
lat1 = 60
lat2 = 90

In [9]:
tas = hxr_gfdl_esm4.tas
weights = np.cos(np.deg2rad(tas.lat))
weights.name = "weights"

!!!!!!!!!!!!!!!! COMPUTE BASELINE (1994-2014) TAS !!!!!!!!!!!!!!!!!!!!!!!!

In [10]:
hgtas = hxr_gfdl_esm4.tas.sel(time=slice("1994", "2014"))
hgtas = hgtas.weighted(weights)
hgtas = hgtas.mean(dim=['lon','lat','time'])
hgtas = hgtas.values
print(hgtas) #(287.2) from ferret

[287.16173045]


In [11]:
hatas = hxr_gfdl_esm4.tas.sel(time=slice("1994", "2014"),lat=slice(lat1, lat2))
hatas = hatas.weighted(weights)
hatas = hatas.mean(dim=['lon','lat','time'])
hatas = hatas.values 
print(hatas)#hatas 264.5 from Ferret

[264.3833672]


In [12]:
print(hgtas,hatas) #hatas 264.5 from Ferret

[287.16173045] [264.3833672]


!!!!!!!!!!!!!!! COMPUTE SHORT-TERM (2020-2040) TAS !!!!!!!!!!!!!!!!!!!

In [13]:
fgtas = fxr_gfdl_esm4.tas.sel(time=slice("2020", "2040"))
fgtas = fgtas.weighted(weights)
fgtas = fgtas.mean(dim=['lon','lat','time'])
fgtas_st = fgtas.values
print(fgtas_st) #287.8 from Ferret

[287.75333068]


In [14]:
fatas = fxr_gfdl_esm4.tas.sel(time=slice("2020", "2040"),lat=slice(lat1, lat2))
fatas = fatas.weighted(weights)
fatas = fatas.mean(dim=['lon','lat','time'])
fatas_st = fatas.values
print(fatas_st) #265.6 From ferret

[265.49362442]


!!!!!!!!!!!!!!!! COMPUTE LONG-TERM (2080-2100) TAS !!!!!!!!!!!!!!!!!!!

In [15]:
fgtas_lt = fxr_gfdl_esm4.tas.sel(time=slice("2080", "2100"))
fgtas_lt = fgtas_lt.weighted(weights)
fgtas_lt = fgtas_lt.mean(dim=['lon','lat','time'])
fgtas_lt = fgtas_lt.values
print(fgtas_lt) #289.8 from Ferret

[289.80264889]


In [16]:
fatas_lt = fxr_gfdl_esm4.tas.sel(time=slice("2080", "2100"),lat=slice(lat1, lat2))
fatas_lt = fatas_lt.weighted(weights)
fatas_lt = fatas_lt.mean(dim=['lon','lat','time'])
fatas_lt = fatas_lt.values
print(fatas_lt) #269.8 from Ferret

[269.77492881]


!!!!!!!!!!!!!!!!!!!! COMPUTE SHORT-TERM POLAR AMP !!!!!!!!!!!!!!!!!!!!

GLOBAL WARMING

In [17]:
g_tdiff_st = fgtas_st - hgtas
g_tdiff_st

array([0.59160023])

ARCTIC WARMING

In [18]:
a_tdiff_st = fatas_st - hatas
a_tdiff_st

array([1.11025722])

! POLAR AMPLIFICATION SHORT-TERM (1.1/0.6=1.83)

In [19]:
pamp_st = a_tdiff_st / g_tdiff_st
pamp_st

array([1.87670178])

!!!!!!!!!!!!!!!!!!!! COMPUTE LONG-TERM POLAR AMP !!!!!!!!!!!!!!!!!!!!!

In [20]:
#GLOBAL WARMING
print(fgtas_lt,hgtas)
g_tdiff_lt = fgtas_lt - hgtas
g_tdiff_lt #2.64

[289.80264889] [287.16173045]


array([2.64091844])

In [21]:
#ARCTIC WARMING
a_tdiff_lt = fatas_lt - hatas
a_tdiff_lt #5.391561607460574


array([5.39156161])

! POLAR AMPLIFICATION LONG-TERM (5.3/2.6=2.04)

In [22]:
pamp_lt = a_tdiff_lt / g_tdiff_lt
pamp_lt #2.0415479396985075

array([2.04154794])

In [23]:
print("POLAR AMP SHORT TERM:", pamp_st) #1.8 from Ferret
print("POLAR AMP LONG TERM:", pamp_lt) #2.0 from Ferret

POLAR AMP SHORT TERM: [1.87670178]
POLAR AMP LONG TERM: [2.04154794]
